# Using PC Camera 

In [10]:
import cv2 as cv
import numpy as np
cv2=cv
capture = cv.VideoCapture(0)

mycolors = [[17,122,128,43,255,255],
           [67,84,94,179,255,255]]


color_values = [[0,192,255],[0,0,255]]



myPoints = []      #[x,y,colorid]
def findColor(img,mycolors,color_values):
    imgHSV = cv.cvtColor(img,cv.COLOR_BGR2HSV)
    count = 0
    newpoints=[]
    for color in mycolors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv.inRange(imgHSV,lower,upper)
        x,y = getContours(mask)
        cv.circle(imgresult,(x,y),10,color_values[count],cv.FILLED)
        if x!=0 and y!=0:
            newpoints.append([x,y,count])
        count+=1
    return newpoints
#         cv2.imshow(str(color[0]),mask)
        
        
def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h=0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
#             cv2.drawContours(imgresult, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx) 
    return x+w//2,y
            
def drawCanvas(myPoints,color_values):
    for points in myPoints:
        cv.circle(imgresult,(points[0],points[1]),10,color_values[points[2]],cv.FILLED)
        
        
while(1):
    istrue, img = capture.read()
    if istrue == False:
        break
    img = cv2.flip(img, 1)
    
    imgresult = img.copy()
    newpoints = findColor(img,mycolors,color_values)
    if len(newpoints)!=0 :
        for newp in newpoints:
            myPoints.append(newp)
    if len(myPoints)!=0:
        drawCanvas(myPoints,color_values)
        
    cv.imshow('video',imgresult)
    
    if cv.waitKey(2) == 27:
        break
capture.release()        
cv.destroyAllWindows()

# Using web Cam on phone

In [1]:
import requests
import cv2
cv=cv2
import numpy as np
import imutils
  
url = "http://100.66.37.226:8080/shot.jpg"



mycolors = [[17,122,128,43,255,255],
           [67,84,94,179,255,255]]


color_values = [[0,192,255],[0,0,255]]



myPoints = []      #[x,y,colorid]
def findColor(img,mycolors,color_values):
    imgHSV = cv.cvtColor(img,cv.COLOR_BGR2HSV)
    count = 0
    newpoints=[]
    for color in mycolors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv.inRange(imgHSV,lower,upper)
        x,y = getContours(mask)
        cv.circle(imgresult,(x,y),10,color_values[count],cv.FILLED)
        if x!=0 and y!=0:
            newpoints.append([x,y,count])
        count+=1
    return newpoints
#         cv2.imshow(str(color[0]),mask)
        
        
def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h=0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
#             cv2.drawContours(imgresult, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx) 
    return x+w//2,y
            
def drawCanvas(myPoints,color_values):
    for points in myPoints:
        cv.circle(imgresult,(points[0],points[1]),10,color_values[points[2]],cv.FILLED)
        
        
        
        
while True:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)
    
    imgresult = img.copy()
    newpoints = findColor(img,mycolors,color_values)
    if len(newpoints)!=0 :
        for newp in newpoints:
            myPoints.append(newp)
    if len(myPoints)!=0:
        drawCanvas(myPoints,color_values)
        
    cv.imshow('video',imgresult)
  
    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break
  
cv2.destroyAllWindows()

In [1]:
import cv2 as cv
cv2 = cv
import numpy as np


def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv.cvtColor( imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver





capture = cv.VideoCapture(0)

def empty(self):
    pass


cv.namedWindow("TrackBars")
cv.resizeWindow("TrackBars",640,240)
cv.createTrackbar("Hue Min","TrackBars",17,179,empty)
cv.createTrackbar("Hue Max","TrackBars",43,179,empty)
cv.createTrackbar("Sat Min","TrackBars",122,255,empty)
cv.createTrackbar("Sat Max","TrackBars",255,255,empty)
cv.createTrackbar("Val Min","TrackBars",128,255,empty)
cv.createTrackbar("Val Max","TrackBars",255,255,empty)


while(1):
    istrue, img = capture.read()
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv2.inRange(imgHSV,lower,upper)
    imgResult = cv2.bitwise_and(img,img,mask=mask)

    
    imgStack = stackImages(0.6,([img,imgHSV],[mask,imgResult]))
    cv2.imshow("Stacked Images", imgStack)
 
    
    if cv.waitKey(2) == 27:
        break
capture.release()        
cv.destroyAllWindows()